# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:**

The Encoder is a pretrained network with a linear layer at the end. The linear layer creates an ouput of the size embed_size so that the features can be used as first input of the RNN-part of the architectue. I added an extra layer for batch normalization after the given encoder to ensure the values to be not to large when feeding them into the decoder (unnormalized values can increase training time).

The decoder is a RNN network. Like mentioned in a couple of videos from this course, an RNN excels when the input vaule is time dependent. This is obviously the case if the network has to describe an image, because every word depends on the word that has been said before. After going through an LSTM layer the ouput is mapped to the dictionary  size of the vocabulary to predict the next word in the sentence. At my first try's i added a softmax to get the value that should be picked from the dictionary. That did not work out well. Fortunately  i found a forum post that mentioned that the CrossEntropyLoss (used in this notebook) has a embedded softmax. After correcting this issue the model turned out to work way better.

I started with an embedding size of 512 and a word threshold of 5, as suggested in [this paper](https://arxiv.org/pdf/1411.4555.pdf). Later i decreased the embedding size to 256 to increase the traing speed of the model. The number of the hidden size was set to 512 and the batch size was chosen to be 128.

The model only had a significant loss during the first epoch. Furthermore, the model safed after the first epoch performend best at the given pictures in notebook3. The weigts that had been trained for two and three epochs seemed to overfit the data because they often created a sentence that had nothing to do with the actual picture. Therfore the model only trained for one epoch was used to complete the third notebook.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:**

I did not change the transform_train. The transformations are somehow similar to the transformations created in the first project and seem to be a good choise for data augmentation (helps the network not to overfit) and preprocessing (crop it to expected size).

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:**

Since the ResNet Network is an already trained network i thought that updating the weights would only cause more computational power without returning significant results. However, the last layer of the encoder was added and should be trained for good results. 
The complete RNN was not trained before. Due to that fact, all weights had to be updated with backprop to get useful result.


### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**

My first try's used an SGD as suggested in [this paper](https://arxiv.org/pdf/1502.03044.pdf). Later on, i changed to an Adam optimizer. Adam is known the combine the past parts of multiple optimizers like SGD and RMSProp.

In [ ]:
import nltk
nltk.download('punkt')

In [11]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5      # minimum word count threshold
vocab_from_file = True   # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 250          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.01)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 389/414113 [00:00<01:46, 3887.80it/s]

  0%|          | 816/414113 [00:00<01:43, 3992.91it/s]

  0%|          | 1251/414113 [00:00<01:40, 4092.39it/s]

  0%|          | 1722/414113 [00:00<01:36, 4259.28it/s]

  1%|          | 2170/414113 [00:00<01:35, 4322.14it/s]

  1%|          | 2565/414113 [00:00<01:37, 4202.14it/s]

  1%|          | 3013/414113 [00:00<01:36, 4279.85it/s]

  1%|          | 3435/414113 [00:00<01:36, 4260.94it/s]

  1%|          | 3879/414113 [00:00<01:35, 4310.60it/s]

  1%|          | 4296/414113 [00:01<01:37, 4210.50it/s]

  1%|          | 4755/414113 [00:01<01:34, 4315.51it/s]

  1%|▏         | 5199/414113 [00:01<01:33, 4351.95it/s]

  1%|▏         | 5638/414113 [00:01<01:33, 4361.08it/s]

  1%|▏         | 6094/414113 [00:01<01:32, 4415.97it/s]

  2%|▏         | 6552/414113 [00:01<01:31, 4463.61it/s]

  2%|▏         | 7019/414113 [00:01<01:30, 4521.88it/s]

  2%|▏         | 7475/414113 [00:01<01:29, 4

 28%|██▊       | 114507/414113 [00:26<01:05, 4554.33it/s]

 28%|██▊       | 114963/414113 [00:26<01:05, 4540.12it/s]

 28%|██▊       | 115418/414113 [00:26<01:07, 4453.62it/s]

 28%|██▊       | 115880/414113 [00:26<01:06, 4499.70it/s]

 28%|██▊       | 116348/414113 [00:26<01:05, 4551.36it/s]

 28%|██▊       | 116804/414113 [00:26<01:06, 4470.71it/s]

 28%|██▊       | 117252/414113 [00:26<01:06, 4463.51it/s]

 28%|██▊       | 117699/414113 [00:26<01:07, 4367.37it/s]

 29%|██▊       | 118137/414113 [00:26<01:08, 4294.21it/s]

 29%|██▊       | 118606/414113 [00:27<01:07, 4404.34it/s]

 29%|██▉       | 119079/414113 [00:27<01:05, 4495.40it/s]

 29%|██▉       | 119537/414113 [00:27<01:05, 4518.62it/s]

 29%|██▉       | 119990/414113 [00:27<01:05, 4459.25it/s]

 29%|██▉       | 120437/414113 [00:27<01:07, 4357.44it/s]

 29%|██▉       | 120878/414113 [00:27<01:07, 4370.15it/s]

 29%|██▉       | 121324/414113 [00:27<01:06, 4394.06it/s]

 29%|██▉       | 121764/414113 [00:27<01:12, 4029.86it/s

 55%|█████▍    | 226796/414113 [00:52<00:41, 4492.86it/s]

 55%|█████▍    | 227253/414113 [00:52<00:41, 4514.26it/s]

 55%|█████▍    | 227722/414113 [00:52<00:40, 4564.09it/s]

 55%|█████▌    | 228180/414113 [00:52<00:40, 4550.94it/s]

 55%|█████▌    | 228636/414113 [00:52<00:41, 4501.46it/s]

 55%|█████▌    | 229087/414113 [00:52<00:41, 4416.07it/s]

 55%|█████▌    | 229530/414113 [00:52<00:41, 4395.29it/s]

 56%|█████▌    | 229971/414113 [00:52<00:41, 4386.38it/s]

 56%|█████▌    | 230424/414113 [00:52<00:41, 4426.14it/s]

 56%|█████▌    | 230888/414113 [00:52<00:40, 4485.37it/s]

 56%|█████▌    | 231340/414113 [00:53<00:40, 4494.24it/s]

 56%|█████▌    | 231791/414113 [00:53<00:40, 4497.99it/s]

 56%|█████▌    | 232242/414113 [00:53<00:41, 4403.30it/s]

 56%|█████▌    | 232683/414113 [00:53<00:41, 4354.62it/s]

 56%|█████▋    | 233120/414113 [00:53<00:41, 4353.50it/s]

 56%|█████▋    | 233568/414113 [00:53<00:41, 4388.24it/s]

 57%|█████▋    | 234008/414113 [00:53<00:41, 4375.83it/s

 82%|████████▏ | 337760/414113 [01:17<00:17, 4248.88it/s]

 82%|████████▏ | 338186/414113 [01:18<00:17, 4228.05it/s]

 82%|████████▏ | 338616/414113 [01:18<00:17, 4249.28it/s]

 82%|████████▏ | 339042/414113 [01:18<00:17, 4221.33it/s]

 82%|████████▏ | 339485/414113 [01:18<00:17, 4280.44it/s]

 82%|████████▏ | 339918/414113 [01:18<00:17, 4293.81it/s]

 82%|████████▏ | 340375/414113 [01:18<00:16, 4372.00it/s]

 82%|████████▏ | 340813/414113 [01:18<00:16, 4341.00it/s]

 82%|████████▏ | 341248/414113 [01:18<00:16, 4342.16it/s]

 83%|████████▎ | 341683/414113 [01:18<00:16, 4321.96it/s]

 83%|████████▎ | 342122/414113 [01:18<00:16, 4339.61it/s]

 83%|████████▎ | 342562/414113 [01:19<00:16, 4355.10it/s]

 83%|████████▎ | 342998/414113 [01:19<00:16, 4310.68it/s]

 83%|████████▎ | 343433/414113 [01:19<00:16, 4319.47it/s]

 83%|████████▎ | 343875/414113 [01:19<00:16, 4347.84it/s]

 83%|████████▎ | 344310/414113 [01:19<00:16, 4340.77it/s]

 83%|████████▎ | 344748/414113 [01:19<00:15, 4349.01it/s

Done (t=1.11s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
## NOTE:
## Training here was endend after 1 Epoch with a keyboard interrupt
## Next cell continuess with epoch 2-3



import torch.utils.data as data
import numpy as np
import os
import requests
import time


# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 3.0337, Perplexity: 20.7746
Epoch [1/3], Step [200/3236], Loss: 2.7760, Perplexity: 16.05507
Epoch [1/3], Step [300/3236], Loss: 2.7974, Perplexity: 16.4026
Epoch [1/3], Step [400/3236], Loss: 2.4960, Perplexity: 12.1334
Epoch [1/3], Step [500/3236], Loss: 2.8706, Perplexity: 17.6470
Epoch [1/3], Step [600/3236], Loss: 2.4673, Perplexity: 11.7901
Epoch [1/3], Step [700/3236], Loss: 2.8743, Perplexity: 17.7128
Epoch [1/3], Step [800/3236], Loss: 2.3923, Perplexity: 10.9389
Epoch [1/3], Step [900/3236], Loss: 2.2838, Perplexity: 9.81347
Epoch [1/3], Step [1000/3236], Loss: 2.3644, Perplexity: 10.6380
Epoch [1/3], Step [1100/3236], Loss: 2.5135, Perplexity: 12.3485
Epoch [1/3], Step [1200/3236], Loss: 2.3850, Perplexity: 10.8591
Epoch [1/3], Step [1300/3236], Loss: 2.4198, Perplexity: 11.2441
Epoch [1/3], Step [1400/3236], Loss: 2.7199, Perplexity: 15.1783
Epoch [1/3], Step [1500/3236], Loss: 2.9517, Perplexity: 19.1385
Epoch [1/3], Step [1600/3236], Lo

In [15]:
## NOTE:
## Training for epoch 2,3



import torch.utils.data as data
import numpy as np
import os
import requests
import time




# Load pre-trained weights before resuming training.
encoder_file = "encoder-1.pkl"
decoder_file = "decoder-1.pkl"
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))


# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch+1, num_epochs+1, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
        # Save the weights.
        if i_step % print_every == 0:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder_Inside_loop.pkl'))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder_Inside_loop.pkl'))
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder_resume-%d.pkl' % int(epoch+1)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder_resume-%d.pkl' % int(epoch+1)))

# Close the training log file.
f.close()

Epoch [2/3], Step [250/3236], Loss: 2.7229, Perplexity: 15.2240
Epoch [2/3], Step [500/3236], Loss: 2.2974, Perplexity: 9.94847
Epoch [2/3], Step [750/3236], Loss: 2.1322, Perplexity: 8.433379
Epoch [2/3], Step [1000/3236], Loss: 2.2617, Perplexity: 9.5993
Epoch [2/3], Step [1250/3236], Loss: 2.1690, Perplexity: 8.74962
Epoch [2/3], Step [1500/3236], Loss: 2.1777, Perplexity: 8.82591
Epoch [2/3], Step [1750/3236], Loss: 3.2536, Perplexity: 25.8845
Epoch [2/3], Step [2000/3236], Loss: 2.2955, Perplexity: 9.92995
Epoch [2/3], Step [2250/3236], Loss: 2.0630, Perplexity: 7.86985
Epoch [2/3], Step [2500/3236], Loss: 2.3397, Perplexity: 10.3778
Epoch [2/3], Step [2750/3236], Loss: 2.3296, Perplexity: 10.2739
Epoch [2/3], Step [3000/3236], Loss: 2.1267, Perplexity: 8.387207
Epoch [3/3], Step [250/3236], Loss: 2.0407, Perplexity: 7.696202
Epoch [3/3], Step [500/3236], Loss: 2.1577, Perplexity: 8.65159
Epoch [3/3], Step [750/3236], Loss: 2.0934, Perplexity: 8.11221
Epoch [3/3], Step [1000/3236]

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [16]:
# (Optional) TODO: Validate your model.